In [1]:
#RUN ONLY ONCE
!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import random
import ctypes  # An included library with Python install.
def Mbox(title, text, style):
    return ctypes.windll.user32.MessageBoxW(0, text, title, style)

#Instructions: Fill in your details in the config, run the script, solve the captcha, and wait.

#CONFIGURATION
CHROME_DRIVER_DOWNLOADED_PATH = 'C:/Users/username/Desktop/selenium/chromedriver.exe'
GLOBAL_SLOWDOWN = 1 #Do not edit.
CAPTCHA_WAIT = 100 #Means you have 100 seconds to do the captcha
REFRESH_WAIT = random.randint(269, 342) #If you refresh too quickly the site will block you for a few hours.
OFFICE_NAME = "OAKLAND COLISEUM"
# OFFICE_NAME = "OAKLAND CLAREMONT"
# OFFICE_NAME = "HAYWARD"
# OFFICE_NAME = "EL CERRITO"
FIRST_NAME = "BOB"
LAST_NAME = "JONES"
DL_NUMBER = "Y1234567"
BIRTH_MONTH = "1" #2 digits
BIRTH_DAY = "1" #2 digits
BIRTH_YEAR = "1996"
TEL_AREA = "510"
TEL_PREFIX = "123"
TEL_SUFFIX = "2345"
GMAIL = "abcdef@gmail.com"
GMAIL_PASS = "P@ssword!"
EMAIL_TARGET_ADDRESSES = ['yourself@gmail.com', 'yourmum@gmail.com']

#Used to prompt the site. Enter a valid weekday after today.
NEXT_WEEKDAY_DAY_MONTH_YEAR = 'THU, 19 Sep 2019'

#Enter 1 if you're running on windows for the popup, otherwise enter 0
WINDOWS = 0

#Optional:
#Months available to trigger alert, enter exactly 2 months in same format
appointment_date_alert_filter = ['Sep', 'Oct']
timecheck = '1:00 PM'

#Remember to enable less secure login via gmail settings to allow this to work.
#Copy file and run different OFFICE_NAME(s) on different tabs to run them simultaneously
##############################################################################################################################
driver = webdriver.Chrome(executable_path = CHROME_DRIVER_DOWNLOADED_PATH)
driver.get('https://www.dmv.ca.gov/wasapp/foa/driveTest.do')
assert "Make Appointment" in driver.title

#SET DMV OFFICE
element = driver.find_element_by_name('officeId')
element.send_keys(OFFICE_NAME)
time.sleep(GLOBAL_SLOWDOWN)

#SET TEST TYPE AS CAR
radioElement = driver.find_element_by_id("DT")
radioElement.click()
time.sleep(GLOBAL_SLOWDOWN)

#SET FIRST NAME
element = driver.find_element_by_name('firstName')
element.send_keys(FIRST_NAME)
time.sleep(GLOBAL_SLOWDOWN)

#SET LAST NAME
element = driver.find_element_by_name('lastName')
element.send_keys(LAST_NAME)
time.sleep(GLOBAL_SLOWDOWN)

#SET PERMIT LICENSE NUMBER
element = driver.find_element_by_name('dlNumber')
element.send_keys(DL_NUMBER)
time.sleep(GLOBAL_SLOWDOWN)

#SET BIRTH DATE MM
element = driver.find_element_by_name('birthMonth')
element.send_keys(BIRTH_MONTH)
time.sleep(GLOBAL_SLOWDOWN)
#SET BIRTH DATE DD
element = driver.find_element_by_name('birthDay')
element.send_keys(BIRTH_DAY)
#SET BIRTH DATE YYYY
time.sleep(GLOBAL_SLOWDOWN)
element = driver.find_element_by_name('birthYear')
element.send_keys(BIRTH_YEAR)

time.sleep(GLOBAL_SLOWDOWN)
#SET PHONE AREA CODE
element = driver.find_element_by_name('telArea')
element.send_keys(TEL_AREA)
time.sleep(GLOBAL_SLOWDOWN)
#SET PHONE NEXT 3 DIGITS
element = driver.find_element_by_name('telPrefix')
element.send_keys(TEL_PREFIX)
time.sleep(GLOBAL_SLOWDOWN)
#SET PHONE LAST 4 DIGITS
element = driver.find_element_by_name('telSuffix')
element.send_keys(TEL_SUFFIX)

time.sleep(GLOBAL_SLOWDOWN)

#NEXT PAGE
driver.find_element_by_xpath("//input[@type='submit' and @value='Continue']").click()

time.sleep(CAPTCHA_WAIT)
count = 0

while (1):
    flag = 0
    print('try', count)
    try:
        element = driver.find_element_by_css_selector('input.ng_cal_input_field')
        element.send_keys(NEXT_WEEKDAY_DAY_MONTH_YEAR)
        element = driver.find_element_by_name('requestedTime')
        element.send_keys(timecheck)
        time.sleep(GLOBAL_SLOWDOWN)
        driver.find_element_by_xpath("//input[@type='submit' and @value='Check for Availability']").click()
        el=driver.find_element_by_css_selector("table > tbody td[data-title='Appointment']")  
        print(el.text)
    except KeyboardInterrupt:
        raise
    except:
        element.send_keys(Keys.F5)
        print('Unable to find date')
        time.sleep(REFRESH_WAIT)
        continue
    time.sleep(REFRESH_WAIT)
    count+=1
    for month in appointment_date_alert_filter:
        if month in el.text:
            flag = 1
            break
    time.sleep(GLOBAL_SLOWDOWN)


import smtplib

if flag == 1:
    print('sending email')
    gmail_user = GMAIL
    gmail_password = GMAIL_PASS

    sent_from = GMAIL
    to = EMAIL_TARGET_ADDRESSES
    subject = 'DMV Appointment Available'
    body = 'Get the fk up and sign up for the driving test now \n' + el.text

    email_text = """\
    From: %s \n\n
    To: %s \n\n
    Subject: %s \n\n

    %s
    """ % (sent_from, ", ".join(to), subject, body)

    try:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.ehlo()
        server.login(gmail_user, gmail_password)
        time.sleep(2)
        server.sendmail(sent_from, to, email_text)
        server.close()
        if windows:
            Mbox('BOOK DMV APPOINTMENT', 'APPOINTMENT FOUND ' + el.text, 1)
        print ('Email sent!')
    except:
        print ('Something went wrong...')

driver.close()
